# Case study 1: Simulation with a SARS-CoV-2 Model

In [97]:
import numpy as np

from src.eliater import version_df
from src.eliater.discover_latent_nodes import find_nuisance_variables, remove_nuisance_variables
#from src.eliater.examples import sars_cov_2_example_discrete as example
from src.eliater.examples.sars_cov2 import sars_large_example as example
from src.eliater.examples.sars_cov2 import generate_continuous
from src.eliater.network_validation import print_graph_falsifications
from y0.algorithm.estimation import estimate_ace
from y0.algorithm.identify import Identification, identify_outcomes
from y0.dsl import P, Variable

version_df()

,key,value
0,eliater,0.0.1-dev-9b2684c9
1,y0,0.2.5-9b2684c9
2,Run at,2024-01-20 18:08:43


In [98]:
treatment = Variable("EGFR")
outcome = Variable("cytok")
SEED = 100

This is case study 1 in Figure 6 in this paper: Eliater: an open source software for causal query estimation from observational measurements of biomolecular networks. The figure below shows the SARS-CoV-2 network (Mohammad-Taheri et al., 2022; Zucker
et al., 2021), which models the activation of Cytokine Release Syndrome (Cytokine Storm), a known factor causing tissue damage in severely ill SARS-CoV-2 patients (Ulhaq and Soraya, 2020).

In [99]:
graph = example.graph

This case study used synthetic observational data. The generation of this synthetic data was inspired by common biological practices. For each endogenous variable $X$ including $EGFR$, we represented  biomolecular reactions using Hill equations \cite{alon2019introduction}. Specifically, we generated observations of each node $X$ from a Binomial distribution with probability of $\frac{1}{1 + \exp(\mathbf{\theta}^{\prime} Pa(X) + \theta_0)}$, where $Pa(X)$ is a $q \times 1$ vector of measurements related to the parent of $X$, $\mathbf{\theta}'$ is a $1 \times q$ parameter vector, and $\theta_0$ is a scalar. The probability distributions for the exogenous variables were simulated from a Binomial distribution with a random probability between 0.4 to 0.8.

In [101]:
# get observational data
#data = example.generate_data(1000, seed=SEED)
data = generate_continuous(1000, seed=SEED)
data.head()

,SARS_COV2,ACE2,Ang,AGTR1,ADAM17,Toci,Sil6R,EGF,TNF,Gefi,EGFR,PRR,NFKB,IL6STAT3,IL6AMP,cytok
0,73.679109,17.576840,93.861006,101.151905,98.540180,56.005886,69.424218,96.505141,99.296894,33.424504,1,97.370407,29.751989,29.366957,38.082656,59.631856
1,56.620967,31.727878,78.949305,99.479725,99.297518,56.022003,64.051996,98.542970,98.511394,47.897558,1,89.538987,38.158193,22.415073,37.129663,56.431981
2,65.013230,24.644308,89.730900,100.509177,99.437188,49.753317,82.287237,91.929421,99.807326,52.808541,1,94.206983,34.403547,44.666545,52.566070,77.488187
3,76.249522,19.484148,98.286645,98.466683,98.528750,55.121697,74.968228,97.613234,99.470404,50.439736,1,97.493690,29.059414,37.505798,41.889477,63.836998
4,76.961345,18.504758,92.974745,99.686780,97.174635,45.153245,55.506413,95.726874,99.440437,35.386174,1,98.792294,32.300719,19.319097,33.277613,52.521325


## Step 1: Verify correctness of the network structure

In [70]:
print_graph_falsifications(
    graph=graph, data=data, method="chi-square", verbose=True, significance_level=0.01
)

All 99 d-separations implied by the network's structure are consistent with the data, meaning that none of the data-driven conditional independency tests' null hypotheses were rejected at p<0.01.

Finished in 47.54 seconds.


,left,right,given,stats,p,dof,p_adj,p_adj_significant
0,AGTR1,SARS_COV2,Ang,1.233919,0.539582,2,1.0,False
1,IL6STAT3,NFKB,ADAM17|EGFR,3.506329,0.476917,4,1.0,False
2,ACE2,EGF,Ang,0.788424,0.674211,2,1.0,False
3,EGF,IL6STAT3,ADAM17,0.435672,0.804257,2,1.0,False
4,SARS_COV2,TNF,Ang,5.257191,0.072180,2,1.0,False
...,...,...,...,...,...,...,...,...
94,SARS_COV2,Toci,,1.052676,0.304892,1,1.0,False
95,NFKB,Sil6r,ADAM17,0.329299,0.848191,2,1.0,False
96,Gefi,Toci,,1.271984,0.259395,1,1.0,False
97,IL6AMP,PRR,IL6STAT3|NFKB,0.561504,0.755216,2,1.0,False


None of the tests failed for this graph. The reason is that the data is simulated according to the graph structure, hence we expect the d-separations implied by the network be validated by the data. 

## Step 2: Check query identifiability

In [104]:
identify_outcomes(graph=graph, treatments=treatment, outcomes=outcome)

Sum[ACE2, ADAM17, AGTR1, Ang, IL6AMP, IL6STAT3, NFKB, PRR, SARS_COV2, Sil6r, TNF, Toci](P(ACE2 | SARS_COV2) * P(AGTR1 | ACE2, Ang, SARS_COV2) * P(IL6AMP | ACE2, ADAM17, AGTR1, Ang, EGF, EGFR, Gefi, IL6STAT3, NFKB, PRR, SARS_COV2, Sil6r, TNF, Toci) * P(IL6STAT3 | ACE2, ADAM17, AGTR1, Ang, SARS_COV2, Sil6r, Toci) * P(TNF | ACE2, ADAM17, AGTR1, Ang, SARS_COV2) * P(cytok | ACE2, ADAM17, AGTR1, Ang, EGF, EGFR, Gefi, IL6AMP, IL6STAT3, NFKB, PRR, SARS_COV2, Sil6r, TNF, Toci) * Sum[ACE2, ADAM17, AGTR1, Ang, EGF, EGFR, Gefi, IL6AMP, IL6STAT3, NFKB, PRR, SARS_COV2, Sil6r, TNF, cytok](P(ACE2, ADAM17, AGTR1, Ang, EGF, EGFR, Gefi, IL6AMP, IL6STAT3, NFKB, PRR, SARS_COV2, Sil6r, TNF, Toci, cytok)) * P(ADAM17 | ACE2, AGTR1, Ang, SARS_COV2, Toci) * P(Sil6r | ACE2, ADAM17, AGTR1, Ang, SARS_COV2, Toci) * P(Ang | ACE2, SARS_COV2) * P(SARS_COV2) * P(NFKB | ACE2, ADAM17, AGTR1, Ang, EGF, EGFR, Gefi, PRR, SARS_COV2, TNF) * P(PRR | ACE2, Gefi, SARS_COV2))

The query is identifiable.

## Step 3: Find nuisance variables and mark them as latent

This function finds the nuisance variables for the input graph.

In [79]:
nuisance_variables = find_nuisance_variables(graph, treatments=treatment, outcomes=outcome)
nuisance_variables

set()

No variable is identified as the nuisance variable. Hence the simplified network in the next step will produce a graph similar to the original graph.

## Step 4: Simplify the network

The following function finds the nuisance variables (step 3), marks them as latent and then applies Evan's simplification rules to remove the nuisance variables. As there are no nuisance variables, the new graph will be the same as the original graph.

In [105]:
new_graph = remove_nuisance_variables(graph, treatments=treatment, outcomes=outcome)

## Step 5: Estimate the query

In [106]:
estimate_ace(new_graph, treatments=treatment, outcomes=outcome, data=data)

ValueError: 

## Evaluation criterion
As we used synthetic data set, we were able to generate two interventional data sets where in
one EGFR was set to 1, and the other one EGFR is set to 0. The ATE was calculated by subtracting the average value of Cytokine Storm obtained from each interventional data,
resulting in the ground truth ATE=-0.018. The negative ATE indicates that the Gefitinib drug can reduce the increase in Cytokine Storm levels, hence can help in treating patients having SARS-Cov-2.

In [107]:
def get_background_ace(seed=None) -> float:
    # get interventional data where treatment is set to 1
    data_1 = generate_continuous(1000, {treatment: 1.0}, seed=seed)
    # get interventional data where treatment is set to 0
    data_0 = generate_continuous(1000, {treatment: 0.0}, seed=seed)
    return data_1.mean()[outcome.name] - data_0.mean()[outcome.name]

# get the true value of ATE
get_background_ace(seed=SEED)

0.7965175191822595

In [82]:
# def get_background_ace(seed=None) -> float:
#     # get interventional data where treatment is set to 1
#     data_1 = example.generate_data(1000, {treatment: 1.0}, seed=seed)
#     # get interventional data where treatment is set to 0
#     data_0 = example.generate_data(1000, {treatment: 0.0}, seed=seed)
#     return data_1.mean()[outcome.name] - data_0.mean()[outcome.name]
# 
# # get the true value of ATE
# get_background_ace(seed=SEED)

-0.024999999999999967

jThe estimated $\widehat{\mathrm{ATE}}=-0.2$, comparable in sign and magnitute to the ground truth ATE=-0.02. The discrepancy in the value of results is due to non-linear, and complex data generation procedure, which is similar to the real-life experimental artifacts, and due to the approximate nature of the modeling assumption.

In [65]:
#Relative change
((-0.22+0.018)/-0.018)

1122.2222222222224